In [11]:
import requests
import json
import re
import urllib3 # For disabling SSL warnings

# Disable InsecureRequestWarning: Unverified HTTPS request is being made.
# Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def get_bike_release_code():
    """
    Makes a request to the Corethree API to confirm a bike hire
    and extracts the release code from the JSON response.
    Disables SSL verification for this specific host if it uses a self-signed cert.
    """
    url = "https://ce-a22.corethree.net/Workflows/HandleEventWithNode?format=json"

    headers = {
        "Host": "ce-a22.corethree.net",
        "Accept": "*/*",
        "c3-encoding": "Kv6OJKA1JWRui1R+UltG2iCZBcb3+EMMfBu5aAhZNEXnA3QTJHKcKBLT+Hd097N5",
        "User-Agent": "Core/202503171232 (iOS; iPad14,1; iPadOS 18.3.2; uk.gov.tfl.cyclehire)",
        "Accept-Language": "en-SG,en-GB;q=0.9,en;q=0.8",
    }

    # The Node data is complex. It's best to define it as a multiline string.
    node_data_xml = """<Node Type%3D"Node.FormControls.Button" ID%3D"page_button1" SortOrder%3D"25" TTL%3D"3600" AliasMode%3D"Passive">
<Name>Confirm hire<%2FName>
<TreeMode>Leaf<%2FTreeMode>
<Language><%2FLanguage>
<TargetUri>part%3A%2F%2FClients.TfL.EBikePhase2.ConfirmMemberHire%3FTerminalName%3D300205%26amp%3BPointName%3DCromer Street%2C Bloomsbury%26amp%3BLCHS_Confirm%3D1%26amp%3BnbBikes%3D(null)<%2FTargetUri>
<Tags>
<Tag key%3D"Style.Cell.ForegroundColor">#FFFFFF<%2FTag>
<Tag key%3D"Style.Cell.BorderColor">#EE0000<%2FTag>
<Tag key%3D"Style.Cell.CenterVertically">1<%2FTag>
<Tag key%3D"Style.Cell.TextAlign">center<%2FTag>
<Tag key%3D"Style.Cell.BackgroundBorderRadius">5%<%2FTag>
<Tag key%3D"Style.Cell.Width">70%<%2FTag>
<Tag key%3D"Style.Cell.Margin.BackgroundColor">#FFFFFF<%2FTag>
<Tag key%3D"Style.Cell.BackgroundColor">#EE0000<%2FTag>
<Tag key%3D"Style.Cell.BorderWidth">1px<%2FTag>
<Tag key%3D"Style.Cell.HideNativeWidgets">1<%2FTag>
<Tag key%3D"Style.Cell.Margin">50 40 40 40<%2FTag>
<Tag key%3D"Style.Cell.FontSize">16px<%2FTag>
<Tag key%3D"Style.Class">button_set page_button1<%2FTag>
<Tag key%3D"Style.Cell.FontName">NJFont-Medium<%2FTag>
<%2FTags>
<%2FNode>"""

    payload = {
        "c3-clienttime": "1748480905.359684", # Consider making this dynamic if needed
        "c3-language": "en",
        "c3-applysensitivedatacheck": "y",
        "c3-scalefactor": "2.00",
        "Node": node_data_xml,
        "c3-capabilities": "inlinevouchers,expirytags,bucketpopulation,vzero,creditcall-chipdna,card.io,camera,camera-front,camera-rear,ble-unknown,location-on-wheninuse,londonriders,londonridersphase2r1,londonridersphase3,londonridersphase4,3dsenabled,ebikesphase2,daypass",
        "c3-batterylevel": "-1.000000",
        "c3-userlat": "51.5282",
        "c3-deviceid": "555D91A6-5B1E-49BC-9624-1989B4DA4833", # This might need to be dynamic/unique
        "c3-userlong": "-0.121092",
        "Event": "Click",
        "c3-controlvals": "cHTnp0wCbVOhbs12x8sR4+2I/8CVACvEd8Zn5e3Tpas=", # This might be session specific
        "c3-userauth": "564e7ff6ebbf80c4cafb4c7b7d3ea7bbc4435ad0|bcSxLxDWpaTC" # This is likely an auth token
    }

    response = None # Initialize response to None for broader scope in error handling
    try:
        print(f"Attempting POST request to {url} (SSL verification disabled)")
        # Add verify=False here, and a timeout
        response = requests.post(url, headers=headers, data=payload, verify=False, timeout=15) # Increased timeout slightly
        response.raise_for_status()  # Raise an exception for HTTP errors (4xx or 5xx)
        
        data = response.json()

        # Extract the release code
        # Method 1: Find the child node with the specific name and get its subtitle
        children = data.get("Children", [])
        for child in children:
            if child.get("Name") == "Your cycle hire release code:" and "Subtitle" in child:
                release_code = child.get("Subtitle")
                if release_code:
                    return release_code
        
        # Method 2 (Alternative): Parse from the unlock bar text if Method 1 fails
        for child in children:
            if child.get("ID", "").endswith("_unlockbar") and "Name" in child:
                name_text = child.get("Name", "")
                match = re.search(r"Release code (\d+)", name_text)
                if match:
                    return match.group(1)

        print("Release code not found in the expected structure.")
        # Optionally print the response if the code is not found, for debugging
        # print("Response data:", json.dumps(data, indent=2))
        return None

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
        if response is not None:
             print(f"Response content: {response.text}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err: # More general requests exception
        print(f"An error occurred during the request: {req_err}")
    except json.JSONDecodeError:
        if response is not None:
            print(f"Failed to decode JSON response. Status: {response.status_code}, Response text: {response.text}")
        else:
            print("Failed to decode JSON response, and response object is None.")
    except Exception as e: # Catch any other unexpected errors
        print(f"An unexpected error occurred: {e}")
    
    return None

if __name__ == "__main__":
    print("Attempting to get bike release code from live API...")
    release_code = get_bike_release_code()

    if release_code:
        print(f"Successfully retrieved release code from live API: {release_code}")
    else:
        print("Failed to retrieve release code from live API.")

Attempting to get bike release code from live API...
Attempting POST request to https://ce-a22.corethree.net/Workflows/HandleEventWithNode?format=json (SSL verification disabled)
Successfully retrieved release code from live API: 23313


In [12]:
import requests
import json
import re
import urllib3
import time # To generate fresh client times, though the API might expect the one tied to c3-encoding
from typing import Literal, Dict, Any, Optional, Tuple

# Disable InsecureRequestWarning: Unverified HTTPS request is being made.
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# --- Define Location Data ---
# We'll store the unique parts from your curl commands per location.
# Ideally, c3_encoding and c3_clienttime would be fetched dynamically before this call.
# For this SDK, we'll use the static ones you provided.

LocationKey = Literal[
    "cromer_street",
    "taviton_street",
    "warren_street_station"
]

# Store the data points specific to each location from the cURL commands
# Note: c3_encoding and c3_clienttime are highly likely to be dynamic and session-based.
# Using these static values will only work if the session they belong to is still valid
# or if the server is lenient for these specific example values.
LOCATION_DATA: Dict[LocationKey, Dict[str, str]] = {
    "cromer_street": {
        "terminal_name": "300205",
        "point_name": "Cromer Street, Bloomsbury",
        "c3_encoding": "Kv6OJKA1JWRui1R+UltG2iCZBcb3+EMMfBu5aAhZNEXnA3QTJHKcKBLT+Hd097N5", # From 1st curl
        "c3_clienttime": "1748480905.359684" # From 1st curl
    },
    "taviton_street": {
        "terminal_name": "001009",
        "point_name": "Taviton Street, Bloomsbury",
        "c3_encoding": "hjQd5cl1SN7BOdmflRPMZwu1UnranBQaYc1W+u/ofJSmJa24Ca9fbkVYjg5SZ+Lg", # From 2nd curl
        "c3_clienttime": "1748481522.599196" # From 2nd curl
    },
    "warren_street_station": {
        "terminal_name": "001090",
        "point_name": "Warren Street Station, Euston",
        "c3_encoding": "Af1F2GlMLbIbykRF6YQQbhJQxCWXYsXyOdUx4M2KxIAvFtrFbaK3CmUhY1dwxDa0", # From 3rd curl
        "c3_clienttime": "1748481544.979739" # From 3rd curl
    }
}

class TflCycleHireSDK:
    BASE_URL = "https://ce-a22.corethree.net/Workflows/HandleEventWithNode?format=json"
    
    # Common values from the cURL requests
    DEFAULT_USER_AGENT = "Core/202503171232 (iOS; iPad14,1; iPadOS 18.3.2; uk.gov.tfl.cyclehire)"
    DEFAULT_C3_LANGUAGE = "en"
    DEFAULT_C3_APPLYSENSITIVEDATACHECK = "y"
    DEFAULT_C3_SCALEFACTOR = "2.00"
    DEFAULT_C3_CAPABILITIES = "inlinevouchers,expirytags,bucketpopulation,vzero,creditcall-chipdna,card.io,camera,camera-front,camera-rear,ble-unknown,location-on-wheninuse,londonriders,londonridersphase2r1,londonridersphase3,londonridersphase4,3dsenabled,ebikesphase2,daypass"
    DEFAULT_C3_BATTERYLEVEL = "-1.000000"
    DEFAULT_C3_USERLAT = "51.5282" # These seem static in your examples
    DEFAULT_C3_USERLONG = "-0.121092" # These seem static in your examples
    DEFAULT_C3_DEVICEID = "555D91A6-5B1E-49BC-9624-1989B4DA4833" # This is often unique per device
    DEFAULT_EVENT = "Click"
    DEFAULT_C3_CONTROLVALS = "cHTnp0wCbVOhbs12x8sR4+2I/8CVACvEd8Zn5e3Tpas=" # Likely session-specific
    DEFAULT_C3_USERAUTH = "564e7ff6ebbf80c4cafb4c7b7d3ea7bbc4435ad0|bcSxLxDWpaTC" # Definitely an auth token

    def __init__(self,
                 user_agent: str = DEFAULT_USER_AGENT,
                 c3_deviceid: str = DEFAULT_C3_DEVICEID,
                 c3_userauth: str = DEFAULT_C3_USERAUTH,
                 c3_controlvals: str = DEFAULT_C3_CONTROLVALS,
                 user_lat: str = DEFAULT_C3_USERLAT,
                 user_long: str = DEFAULT_C3_USERLONG
                 ):
        self.user_agent = user_agent
        self.c3_deviceid = c3_deviceid
        self.c3_userauth = c3_userauth
        self.c3_controlvals = c3_controlvals
        self.user_lat = user_lat
        self.user_long = user_long
        self.session = requests.Session() # Use a session for potential cookie handling, keep-alive

    def _build_node_xml(self, terminal_name: str, point_name: str) -> str:
        # URL encode point_name for the TargetUri (spaces become %20, etc.)
        # The curl examples use `+` for spaces in PointName and also `&` for `&`
        # The TargetUri also has some parts already %-encoded.
        # We need to be careful here:
        # part%3A%2F%2F becomes part://
        # %3F becomes ?
        # %3D becomes =
        # %26amp%3B becomes & (after first decoding & to & then & to %26)
        # Let's replicate the structure carefully.
        # The TargetUri parameters are TerminalName, PointName, LCHS_Confirm, nbBikes

        # PointName needs spaces as '+' and other special chars URL encoded.
        # The curl examples seem to have it pre-encoded in a specific way.
        # For example, "Cromer Street, Bloomsbury"
        # becomes "Cromer+Street%2C+Bloomsbury" if we manually replace space with + and , with %2C
        # However, the example shows "Cromer Street%2C Bloomsbury" where space is NOT a +
        # Let's use the exact PointName format from the curl command if it works.
        # The crucial part is that `requests` will URL-encode the *entire* payload dictionary values,
        # so we need to provide the TargetUri string in a way that, after one layer of
        # URL encoding by `requests`, it matches what the server expects.
        # The curl --data is already URL-encoded.
        # The TargetUri value within the Node XML is *itself* partially URL-encoded.

        # Let's assume the point_name from LOCATION_DATA is already in the desired format.
        # Example from curl: TargetUri>part%3A%2F%2FClients.TfL.EBikePhase2.ConfirmMemberHire%3FTerminalName%3D300205%26amp%3BPointName%3DCromer Street%2C Bloomsbury%26amp%3BLCHS_Confirm%3D1%26amp%3BnbBikes%3D(null)<%2FTargetUri
        # The %2F for / and %3A for : are standard.
        # %26amp%3B is tricky. It's likely `&` then URL-encoded.
        # Let's try to construct it simply first. `requests` will encode the `&`.
        
        # Correct construction for TargetUri values that go into the --data payload
        # which is application/x-www-form-urlencoded
        # Inside the Node XML, the TargetUri string itself has some pre-encoding.
        target_uri_template = "part%3A%2F%2FClients.TfL.EBikePhase2.ConfirmMemberHire%3FTerminalName%3D{terminal_name}%26amp%3BPointName%3D{point_name_encoded}%26amp%3BLCHS_Confirm%3D1%26amp%3BnbBikes%3D(null)"
        
        # The point_name in the curl command seems to only have comma encoded as %2C
        # and spaces are literal spaces.
        # Example: "Cromer Street%2C Bloomsbury" (from your first curl)
        # point_name_encoded = point_name.replace(",", "%2C") # This matches curl
        
        # Let's use the point_name as provided in LOCATION_DATA, assuming it's correctly pre-formatted for the TargetUri
        # For "Cromer Street, Bloomsbury", it becomes "Cromer Street%2C Bloomsbury" in the curl TargetUri
        # For "Taviton Street, Bloomsbury", it becomes "Taviton Street%2C Bloomsbury"
        # For "Warren Street Station, Euston", it becomes "Warren Street Station%2C Euston"
        # So, we only need to ensure the comma is %2C. The LOCATION_DATA point_name is the display name.
        
        point_name_for_uri = point_name.replace(",", "%2C")

        target_uri = target_uri_template.format(terminal_name=terminal_name, point_name_encoded=point_name_for_uri)

        return f"""<Node Type%3D"Node.FormControls.Button" ID%3D"page_button1" SortOrder%3D"25" TTL%3D"3600" AliasMode%3D"Passive">
<Name>Confirm hire<%2FName>
<TreeMode>Leaf<%2FTreeMode>
<Language><%2FLanguage>
<TargetUri>{target_uri}<%2FTargetUri>
<Tags>
<Tag key%3D"Style.Cell.ForegroundColor">#FFFFFF<%2FTag>
<Tag key%3D"Style.Cell.BorderColor">#EE0000<%2FTag>
<Tag key%3D"Style.Cell.CenterVertically">1<%2FTag>
<Tag key%3D"Style.Cell.TextAlign">center<%2FTag>
<Tag key%3D"Style.Cell.BackgroundBorderRadius">5%<%2FTag>
<Tag key%3D"Style.Cell.Width">70%<%2FTag>
<Tag key%3D"Style.Cell.Margin.BackgroundColor">#FFFFFF<%2FTag>
<Tag key%3D"Style.Cell.BackgroundColor">#EE0000<%2FTag>
<Tag key%3D"Style.Cell.BorderWidth">1px<%2FTag>
<Tag key%3D"Style.Cell.HideNativeWidgets">1<%2FTag>
<Tag key%3D"Style.Cell.Margin">50 40 40 40<%2FTag>
<Tag key%3D"Style.Cell.FontSize">16px<%2FTag>
<Tag key%3D"Style.Class">button_set page_button1<%2FTag>
<Tag key%3D"Style.Cell.FontName">NJFont-Medium<%2FTag>
<%2FTags>
<%2FNode>"""

    def get_release_code(self, 
                         location_key: LocationKey,
                         # Explicitly require these as they are highly dynamic
                         c3_encoding: str,
                         c3_clienttime: str,
                         timeout: int = 20
                         ) -> Optional[str]:
        """
        Attempts to get a bike release code for a specified location.

        Args:
            location_key: The key for the location from LOCATION_DATA.
            c3_encoding: The c3-encoding header value for this request.
            c3_clienttime: The c3-clienttime payload value for this request.
            timeout: Request timeout in seconds.

        Returns:
            The release code string if successful, None otherwise.
        """
        if location_key not in LOCATION_DATA:
            print(f"Error: Location key '{location_key}' not found.")
            print(f"Available keys: {', '.join(LOCATION_DATA.keys())}")
            return None

        loc_data = LOCATION_DATA[location_key]
        terminal_name = loc_data["terminal_name"]
        point_name_display = loc_data["point_name"] # This is the human-readable one

        headers = {
            "Host": "ce-a22.corethree.net",
            "Accept": "*/*",
            "c3-encoding": c3_encoding, # Dynamic per request
            "User-Agent": self.user_agent,
            "Accept-Language": "en-SG,en-GB;q=0.9,en;q=0.8",
        }

        node_xml = self._build_node_xml(terminal_name, point_name_display)

        payload = {
            "c3-clienttime": c3_clienttime, # Dynamic per request
            "c3-language": self.DEFAULT_C3_LANGUAGE,
            "c3-applysensitivedatacheck": self.DEFAULT_C3_APPLYSENSITIVEDATACHECK,
            "c3-scalefactor": self.DEFAULT_C3_SCALEFACTOR,
            "Node": node_xml,
            "c3-capabilities": self.DEFAULT_C3_CAPABILITIES,
            "c3-batterylevel": self.DEFAULT_C3_BATTERYLEVEL,
            "c3-userlat": self.user_lat,
            "c3-deviceid": self.c3_deviceid,
            "c3-userlong": self.user_long,
            "Event": self.DEFAULT_EVENT,
            "c3-controlvals": self.c3_controlvals,
            "c3-userauth": self.c3_userauth
        }
        
        response_obj = None
        try:
            print(f"\nAttempting to get release code for: {point_name_display} ({location_key})")
            print(f"Using c3-encoding: {c3_encoding[:10]}...") # Print a snippet
            print(f"Using c3-clienttime: {c3_clienttime}")

            response_obj = self.session.post(
                self.BASE_URL, 
                headers=headers, 
                data=payload, 
                verify=False, # As established before
                timeout=timeout
            )
            response_obj.raise_for_status()
            
            data = response_obj.json()

            children = data.get("Children", [])
            for child in children:
                if child.get("Name") == "Your cycle hire release code:" and "Subtitle" in child:
                    release_code = child.get("Subtitle")
                    if release_code:
                        return release_code
            
            for child in children: # Fallback check
                if child.get("ID", "").endswith("_unlockbar") and "Name" in child:
                    name_text = child.get("Name", "")
                    match = re.search(r"Release code (\d+)", name_text)
                    if match:
                        return match.group(1)

            print(f"Release code not found in the expected structure for {point_name_display}.")
            # print("Full response:", json.dumps(data, indent=2)) # For debugging
            return None

        except requests.exceptions.HTTPError as http_err:
            print(f"HTTP error occurred for {point_name_display}: {http_err}")
            if response_obj is not None:
                 print(f"Response content: {response_obj.text[:500]}...") # Print first 500 chars
        except requests.exceptions.ConnectionError as conn_err:
            print(f"Connection error occurred for {point_name_display}: {conn_err}")
        except requests.exceptions.Timeout as timeout_err:
            print(f"Timeout error occurred for {point_name_display}: {timeout_err}")
        except requests.exceptions.RequestException as req_err:
            print(f"Request error occurred for {point_name_display}: {req_err}")
        except json.JSONDecodeError:
            if response_obj is not None:
                print(f"Failed to decode JSON response for {point_name_display}. Status: {response_obj.status_code}, Response text: {response_obj.text[:500]}...")
            else:
                print(f"Failed to decode JSON response for {point_name_display}, and response object is None.")
        except Exception as e:
            print(f"An unexpected error occurred for {point_name_display}: {e}")
        
        return None

# --- Main execution example ---
if __name__ == "__main__":
    sdk = TflCycleHireSDK()

    # --- Attempt for Cromer Street ---
    # Use the specific c3_encoding and c3_clienttime from its curl example
    cromer_details = LOCATION_DATA["cromer_street"]
    code_cromer = sdk.get_release_code(
        location_key="cromer_street",
        c3_encoding=cromer_details["c3_encoding"],
        c3_clienttime=cromer_details["c3_clienttime"]
    )
    if code_cromer:
        print(f"SUCCESS! Cromer Street Release Code: {code_cromer}")
    else:
        print(f"FAILED to get code for Cromer Street.")

    # --- Attempt for Taviton Street ---
    taviton_details = LOCATION_DATA["taviton_street"]
    code_taviton = sdk.get_release_code(
        location_key="taviton_street",
        c3_encoding=taviton_details["c3_encoding"],
        c3_clienttime=taviton_details["c3_clienttime"]
    )
    if code_taviton:
        print(f"SUCCESS! Taviton Street Release Code: {code_taviton}")
    else:
        print(f"FAILED to get code for Taviton Street.")

    # --- Attempt for Warren Street Station ---
    warren_details = LOCATION_DATA["warren_street_station"]
    code_warren = sdk.get_release_code(
        location_key="warren_street_station",
        c3_encoding=warren_details["c3_encoding"],
        c3_clienttime=warren_details["c3_clienttime"]
    )
    if code_warren:
        print(f"SUCCESS! Warren Street Station Release Code: {code_warren}")
    else:
        print(f"FAILED to get code for Warren Street Station.")
        
    # Example of how "autocompletion" for location_key would work in an IDE
    # If you type `sdk.get_release_code(location_key=)` your IDE supporting Literal types
    # should suggest "cromer_street", "taviton_street", "warren_street_station".
    # For example:
    # my_location: LocationKey = "taviton_street" 
    # sdk.get_release_code(my_location, c3_encoding="...", c3_clienttime="...")


Attempting to get release code for: Cromer Street, Bloomsbury (cromer_street)
Using c3-encoding: Kv6OJKA1JW...
Using c3-clienttime: 1748480905.359684
SUCCESS! Cromer Street Release Code: 23313

Attempting to get release code for: Taviton Street, Bloomsbury (taviton_street)
Using c3-encoding: hjQd5cl1SN...
Using c3-clienttime: 1748481522.599196
SUCCESS! Taviton Street Release Code: 21222

Attempting to get release code for: Warren Street Station, Euston (warren_street_station)
Using c3-encoding: Af1F2GlMLb...
Using c3-clienttime: 1748481544.979739
SUCCESS! Warren Street Station Release Code: 11132


In [13]:
import requests
import json
import re
import urllib3
import time
import logging
from typing import Literal, Dict, Any, Optional, Tuple

# --- SDK Specific Exceptions ---
class TflCycleHireSDKError(Exception):
    """Base exception for TFL Cycle Hire SDK errors."""
    pass

class TflCycleHireAPIError(TflCycleHireSDKError):
    """Raised for API-level errors (e.g., HTTP 4xx, 5xx)."""
    def __init__(self, message, status_code=None, response_text=None):
        super().__init__(message)
        self.status_code = status_code
        self.response_text = response_text

class TflCycleHireDataError(TflCycleHireSDKError):
    """Raised when expected data is not found or malformed in the API response."""
    pass

class TflCycleHireConfigError(TflCycleHireSDKError):
    """Raised for configuration issues (e.g., invalid location key)."""
    pass

# --- Configure Logging ---
# In a real application, this would be configured by the consuming application.
# For this SDK example, we'll set up a basic logger.
logger = logging.getLogger(__name__)
# To see output from the SDK's logger in the example, uncomment and configure:
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')


# --- Define Location Data ---
LocationKey = Literal[
    "cromer_street",
    "taviton_street",
    "warren_street_station"
]

# NOTE: The 'c3_encoding' and 'c3_clienttime' values are highly dynamic and
# session-specific. The values below are from the provided cURL examples and
# are unlikely to work for long. They MUST be refreshed for new requests.
# This SDK currently requires them to be passed for each `get_release_code` call.
LOCATION_DATA: Dict[LocationKey, Dict[str, str]] = {
    "cromer_street": {
        "terminal_name": "300205",
        "point_name": "Cromer Street, Bloomsbury",
        "c3_encoding": "Kv6OJKA1JWRui1R+UltG2iCZBcb3+EMMfBu5aAhZNEXnA3QTJHKcKBLT+Hd097N5",
        "c3_clienttime": "1748480905.359684"
    },
    "taviton_street": {
        "terminal_name": "001009",
        "point_name": "Taviton Street, Bloomsbury",
        "c3_encoding": "hjQd5cl1SN7BOdmflRPMZwu1UnranBQaYc1W+u/ofJSmJa24Ca9fbkVYjg5SZ+Lg",
        "c3_clienttime": "1748481522.599196"
    },
    "warren_street_station": {
        "terminal_name": "001090",
        "point_name": "Warren Street Station, Euston",
        "c3_encoding": "Af1F2GlMLbIbykRF6YQQbhJQxCWXYsXyOdUx4M2KxIAvFtrFbaK3CmUhY1dwxDa0",
        "c3_clienttime": "1748481544.979739"
    }
}

class TflCycleHireSDK:
    """
    A basic SDK to interact with a specific TfL Cycle Hire API endpoint
    for retrieving bike release codes.

    Note: This SDK relies on several dynamic parameters (especially
    `c3_encoding` and `c3_clienttime`) that are likely short-lived and
    session-specific. These must be provided accurately for API calls
    to succeed. The default values for other parameters like `c3_userauth`
    are also based on provided examples and might need to be updated.
    """
    BASE_URL = "https://ce-a22.corethree.net/Workflows/HandleEventWithNode?format=json"
    
    # Default values for common parameters, derived from example cURL requests.
    # These might need to be updated or made configurable if they change.
    DEFAULT_USER_AGENT = "Core/202503171232 (iOS; iPad14,1; iPadOS 18.3.2; uk.gov.tfl.cyclehire)"
    DEFAULT_ACCEPT_LANGUAGE = "en-SG,en-GB;q=0.9,en;q=0.8"
    DEFAULT_C3_LANGUAGE = "en"
    DEFAULT_C3_APPLYSENSITIVEDATACHECK = "y"
    DEFAULT_C3_SCALEFACTOR = "2.00"
    DEFAULT_C3_CAPABILITIES = "inlinevouchers,expirytags,bucketpopulation,vzero,creditcall-chipdna,card.io,camera,camera-front,camera-rear,ble-unknown,location-on-wheninuse,londonriders,londonridersphase2r1,londonridersphase3,londonridersphase4,3dsenabled,ebikesphase2,daypass"
    DEFAULT_C3_BATTERYLEVEL = "-1.000000"
    DEFAULT_C3_USERLAT = "51.5282"
    DEFAULT_C3_USERLONG = "-0.121092"
    DEFAULT_C3_DEVICEID = "555D91A6-5B1E-49BC-9624-1989B4DA4833" # Often unique per device/install
    DEFAULT_EVENT = "Click"
    DEFAULT_C3_CONTROLVALS = "cHTnp0wCbVOhbs12x8sR4+2I/8CVACvEd8Zn5e3Tpas=" # Likely session-specific
    DEFAULT_C3_USERAUTH = "564e7ff6ebbf80c4cafb4c7b7d3ea7bbc4435ad0|bcSxLxDWpaTC" # Critical auth token

    NODE_XML_TEMPLATE = """<Node Type%3D"Node.FormControls.Button" ID%3D"page_button1" SortOrder%3D"25" TTL%3D"3600" AliasMode%3D"Passive">
<Name>Confirm hire<%2FName>
<TreeMode>Leaf<%2FTreeMode>
<Language><%2FLanguage>
<TargetUri>part%3A%2F%2FClients.TfL.EBikePhase2.ConfirmMemberHire%3FTerminalName%3D{terminal_name}%26amp%3BPointName%3D{point_name_encoded}%26amp%3BLCHS_Confirm%3D1%26amp%3BnbBikes%3D(null)<%2FTargetUri>
<Tags>
<Tag key%3D"Style.Cell.ForegroundColor">#FFFFFF<%2FTag>
<Tag key%3D"Style.Cell.BorderColor">#EE0000<%2FTag>
<Tag key%3D"Style.Cell.CenterVertically">1<%2FTag>
<Tag key%3D"Style.Cell.TextAlign">center<%2FTag>
<Tag key%3D"Style.Cell.BackgroundBorderRadius">5%<%2FTag>
<Tag key%3D"Style.Cell.Width">70%<%2FTag>
<Tag key%3D"Style.Cell.Margin.BackgroundColor">#FFFFFF<%2FTag>
<Tag key%3D"Style.Cell.BackgroundColor">#EE0000<%2FTag>
<Tag key%3D"Style.Cell.BorderWidth">1px<%2FTag>
<Tag key%3D"Style.Cell.HideNativeWidgets">1<%2FTag>
<Tag key%3D"Style.Cell.Margin">50 40 40 40<%2FTag>
<Tag key%3D"Style.Cell.FontSize">16px<%2FTag>
<Tag key%3D"Style.Class">button_set page_button1<%2FTag>
<Tag key%3D"Style.Cell.FontName">NJFont-Medium<%2FTag>
<%2FTags>
<%2FNode>"""


    def __init__(self,
                 user_agent: str = DEFAULT_USER_AGENT,
                 c3_deviceid: str = DEFAULT_C3_DEVICEID,
                 c3_userauth: str = DEFAULT_C3_USERAUTH,
                 c3_controlvals: str = DEFAULT_C3_CONTROLVALS,
                 user_lat: str = DEFAULT_C3_USERLAT,
                 user_long: str = DEFAULT_C3_USERLONG,
                 disable_ssl_warnings: bool = True,
                 session: Optional[requests.Session] = None):
        """
        Initializes the TflCycleHireSDK.

        Args:
            user_agent: The User-Agent string for requests.
            c3_deviceid: The c3-deviceid value.
            c3_userauth: The c3-userauth token.
            c3_controlvals: The c3-controlvals value.
            user_lat: User's latitude.
            user_long: User's longitude.
            disable_ssl_warnings: If True, suppresses InsecureRequestWarning for verify=False.
            session: An optional requests.Session object to use.
        """
        self.user_agent = user_agent
        self.c3_deviceid = c3_deviceid
        self.c3_userauth = c3_userauth
        self.c3_controlvals = c3_controlvals
        self.user_lat = user_lat
        self.user_long = user_long
        
        if disable_ssl_warnings:
            urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

        self.session = session or requests.Session()
        self.session.headers["User-Agent"] = self.user_agent # Set common header for session

    def _build_node_xml(self, terminal_name: str, point_name_display: str) -> str:
        """
        Constructs the Node XML payload.
        The point_name_display is the human-readable name, which will be
        formatted for the TargetUri (e.g., commas encoded).
        """
        # In the cURL examples, the PointName in TargetUri has commas as %2C and spaces as literal spaces.
        point_name_encoded = point_name_display.replace(",", "%2C")
        
        return self.NODE_XML_TEMPLATE.format(
            terminal_name=terminal_name,
            point_name_encoded=point_name_encoded
        )

    def get_release_code(self, 
                         location_key: LocationKey,
                         c3_encoding: str,
                         c3_clienttime: str,
                         timeout: int = 20
                         ) -> str:
        """
        Attempts to get a bike release code for a specified location.

        Args:
            location_key: The key for the location from LOCATION_DATA.
            c3_encoding: The c3-encoding header value for this request. This is highly
                         dynamic and must be current.
            c3_clienttime: The c3-clienttime payload value for this request. This is highly
                           dynamic and must be current.
            timeout: Request timeout in seconds.

        Returns:
            The release code string.

        Raises:
            TflCycleHireConfigError: If the location_key is invalid.
            TflCycleHireAPIError: For HTTP errors or issues with the API response format.
            TflCycleHireDataError: If the release code cannot be extracted from the response.
            requests.exceptions.RequestException: For network or request-related issues.
        """
        if location_key not in LOCATION_DATA:
            msg = f"Invalid location key: '{location_key}'. Available keys: {', '.join(LOCATION_DATA.keys())}"
            logger.error(msg)
            raise TflCycleHireConfigError(msg)

        loc_data = LOCATION_DATA[location_key]
        terminal_name = loc_data["terminal_name"]
        point_name_display = loc_data["point_name"]

        headers = {
            "Host": "ce-a22.corethree.net", # Standard for requests, but Host header is good practice
            "Accept": "*/*",
            "c3-encoding": c3_encoding,
            "Accept-Language": self.DEFAULT_ACCEPT_LANGUAGE,
            # User-Agent is set on self.session.headers
        }

        node_xml = self._build_node_xml(terminal_name, point_name_display)

        payload = {
            "c3-clienttime": c3_clienttime,
            "c3-language": self.DEFAULT_C3_LANGUAGE,
            "c3-applysensitivedatacheck": self.DEFAULT_C3_APPLYSENSITIVEDATACHECK,
            "c3-scalefactor": self.DEFAULT_C3_SCALEFACTOR,
            "Node": node_xml,
            "c3-capabilities": self.DEFAULT_C3_CAPABILITIES,
            "c3-batterylevel": self.DEFAULT_C3_BATTERYLEVEL,
            "c3-userlat": self.user_lat,
            "c3-deviceid": self.c3_deviceid,
            "c3-userlong": self.user_long,
            "Event": self.DEFAULT_EVENT,
            "c3-controlvals": self.c3_controlvals,
            "c3-userauth": self.c3_userauth
        }
        
        logger.info(f"Requesting release code for: {point_name_display} ({location_key})")
        logger.debug(f"Using c3-encoding (partial): {c3_encoding[:15]}...")
        logger.debug(f"Using c3-clienttime: {c3_clienttime}")
        logger.debug(f"Payload Node XML (partial): {node_xml[:100]}...")


        response_obj = None
        try:
            response_obj = self.session.post(
                self.BASE_URL, 
                headers=headers, 
                data=payload, 
                verify=False, # SSL verification disabled as established before
                timeout=timeout
            )
            response_obj.raise_for_status() # Raises HTTPError for 4xx/5xx
            
            data = response_obj.json()

        except requests.exceptions.HTTPError as http_err:
            error_message = f"HTTP error for {point_name_display}: {http_err}"
            logger.error(error_message, exc_info=True)
            response_text = http_err.response.text if http_err.response else None
            status_code = http_err.response.status_code if http_err.response else None
            raise TflCycleHireAPIError(error_message, status_code=status_code, response_text=response_text) from http_err
        except requests.exceptions.RequestException as req_err: # Catches ConnectionError, Timeout, etc.
            error_message = f"Request failed for {point_name_display}: {req_err}"
            logger.error(error_message, exc_info=True)
            raise # Re-raise the original requests exception or wrap it
        except json.JSONDecodeError as json_err:
            error_message = f"Failed to decode JSON response for {point_name_display}."
            logger.error(error_message, exc_info=True)
            response_text = response_obj.text if response_obj else None
            status_code = response_obj.status_code if response_obj else None
            raise TflCycleHireAPIError(error_message, status_code=status_code, response_text=response_text) from json_err

        # --- Extract release code ---
        children = data.get("Children", [])
        
        # Primary extraction method
        for child in children:
            if child.get("Name") == "Your cycle hire release code:" and "Subtitle" in child:
                release_code = child.get("Subtitle")
                if release_code:
                    logger.info(f"Successfully extracted release code: {release_code} for {point_name_display}")
                    return release_code
        
        # Alternative extraction method (fallback)
        for child in children:
            if child.get("ID", "").endswith("_unlockbar") and "Name" in child:
                name_text = child.get("Name", "")
                match = re.search(r"Release code (\d+)", name_text)
                if match:
                    release_code = match.group(1)
                    logger.info(f"Extracted release code (alternative method): {release_code} for {point_name_display}")
                    return release_code

        logger.error(f"Release code not found in expected structure for {point_name_display}.")
        logger.debug(f"Full API response for {point_name_display}: {json.dumps(data, indent=2)}")
        raise TflCycleHireDataError(f"Could not find release code in response for {point_name_display}.")

# --- Main execution example ---
if __name__ == "__main__":
    # --- Configure logging for the example ---
    logging.basicConfig(
        level=logging.INFO, # Change to DEBUG for more verbose SDK output
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )
    # --- End logging config ---

    sdk = TflCycleHireSDK()

    locations_to_try: list[LocationKey] = ["cromer_street", "taviton_street", "warren_street_station"]
    
    all_successful = True
    for loc_key in locations_to_try:
        # IMPORTANT: These c3_encoding and c3_clienttime values are static examples.
        # They MUST be refreshed for actual use, as they are short-lived.
        # For this test, we are using the ones provided in LOCATION_DATA.
        loc_details = LOCATION_DATA[loc_key]
        current_c3_encoding = loc_details["c3_encoding"]
        current_c3_clienttime = loc_details["c3_clienttime"]

        try:
            code = sdk.get_release_code(
                location_key=loc_key,
                c3_encoding=current_c3_encoding,
                c3_clienttime=current_c3_clienttime
            )
            logger.info(f"SUCCESS! {LOCATION_DATA[loc_key]['point_name']} Release Code: {code}")
        except TflCycleHireSDKError as e:
            logger.error(f"SDK Error for {LOCATION_DATA[loc_key]['point_name']}: {e}")
            if isinstance(e, TflCycleHireAPIError) and e.response_text:
                logger.error(f"API Response (partial): {e.response_text[:500]}...")
            all_successful = False
        except Exception as e: # Catch any other unexpected errors
            logger.error(f"Unexpected error for {LOCATION_DATA[loc_key]['point_name']}: {e}", exc_info=True)
            all_successful = False
        print("-" * 30) # Separator for readability

    if all_successful:
        logger.info("All specified locations processed successfully with example data.")
    else:
        logger.warning("Some locations failed to process. Check logs for details.")

    # Example of autocompletion for location_key (IDE dependent):
    # selected_location: LocationKey = "cromer_street" # IDE should suggest valid keys

2025-05-29 02:25:08,307 - __main__ - INFO - Requesting release code for: Cromer Street, Bloomsbury (cromer_street)
2025-05-29 02:25:09,943 - __main__ - INFO - Successfully extracted release code: 23313 for Cromer Street, Bloomsbury
2025-05-29 02:25:09,944 - __main__ - INFO - SUCCESS! Cromer Street, Bloomsbury Release Code: 23313
2025-05-29 02:25:09,945 - __main__ - INFO - Requesting release code for: Taviton Street, Bloomsbury (taviton_street)


------------------------------


2025-05-29 02:25:11,376 - __main__ - INFO - Successfully extracted release code: 21222 for Taviton Street, Bloomsbury
2025-05-29 02:25:11,377 - __main__ - INFO - SUCCESS! Taviton Street, Bloomsbury Release Code: 21222
2025-05-29 02:25:11,378 - __main__ - INFO - Requesting release code for: Warren Street Station, Euston (warren_street_station)


------------------------------


2025-05-29 02:25:12,878 - __main__ - INFO - Successfully extracted release code: 11132 for Warren Street Station, Euston
2025-05-29 02:25:12,885 - __main__ - INFO - SUCCESS! Warren Street Station, Euston Release Code: 11132
2025-05-29 02:25:12,885 - __main__ - INFO - All specified locations processed successfully with example data.


------------------------------


In [14]:
# (Keep all the previous SDK code, including imports, LOCATION_DATA, etc.)
# ...

# --- Experimental Functions ---

def run_test_attempt(
    sdk_instance: TflCycleHireSDK,
    test_name: str,
    location_key: LocationKey,
    c3_encoding: str,
    c3_clienttime: str,
    expected_to_succeed: bool = True
) -> bool:
    """Helper function to run a single test attempt and log results."""
    logger.info(f"--- Running Test: {test_name} for {location_key} ---")
    logger.info(f"Using c3_encoding (partial): {c3_encoding[:15]}...")
    logger.info(f"Using c3_clienttime: {c3_clienttime}")
    
    success = False
    try:
        code = sdk_instance.get_release_code(
            location_key=location_key,
            c3_encoding=c3_encoding,
            c3_clienttime=c3_clienttime,
            timeout=10 # Shorter timeout for tests
        )
        logger.info(f"Test '{test_name}': SUCCESS! Code: {code}")
        success = True
    except TflCycleHireSDKError as e:
        logger.warning(f"Test '{test_name}': FAILED as expected or unexpectedly. Error: {e}")
        if isinstance(e, TflCycleHireAPIError) and e.response_text:
            logger.debug(f"API Response (partial): {e.response_text[:200]}...")
        success = False # Explicitly set, though already false if exception
    except Exception as e:
        logger.error(f"Test '{test_name}': UNEXPECTED EXCEPTION. Error: {e}", exc_info=True)
        success = False
    
    if expected_to_succeed and not success:
        logger.error(f"!!! Test '{test_name}' was expected to SUCCEED but FAILED. !!!")
    elif not expected_to_succeed and success:
        logger.error(f"!!! Test '{test_name}' was expected to FAIL but SUCCEEDED. !!!")
    
    logger.info(f"--- Test '{test_name}' for {location_key} Concluded ---")
    return success

def perform_expiry_experiments(sdk: TflCycleHireSDK):
    logger.info("\n" + "="*50 + "\nPERFORMING EXPIRY EXPERIMENTS\n" + "="*50)

    # --- Experiment 1: Immediate Reusability of a c3_encoding/c3_clienttime pair ---
    logger.info("\n--- Experiment 1: Immediate Reusability ---")
    loc_key_ex1 = "cromer_street"
    details_ex1 = LOCATION_DATA[loc_key_ex1]
    original_encoding_ex1 = details_ex1["c3_encoding"]
    original_clienttime_ex1 = details_ex1["c3_clienttime"]

    # First attempt (should work based on previous success)
    run_test_attempt(sdk, "Ex1.1: Initial Use", loc_key_ex1, original_encoding_ex1, original_clienttime_ex1, expected_to_succeed=True)
    
    # Second attempt with the exact same tokens, immediately
    # This is expected to FAIL if tokens are single-use for this specific interaction.
    run_test_attempt(sdk, "Ex1.2: Immediate Reuse (Same Location)", loc_key_ex1, original_encoding_ex1, original_clienttime_ex1, expected_to_succeed=False)

    # Third attempt with same tokens, different location (if Ex1.2 failed, this likely will too)
    loc_key_ex1_alt = "taviton_street"
    if loc_key_ex1_alt != loc_key_ex1: # Ensure it's actually a different location
        run_test_attempt(sdk, "Ex1.3: Immediate Reuse (Different Location)", loc_key_ex1_alt, original_encoding_ex1, original_clienttime_ex1, expected_to_succeed=False)


    # --- Experiment 2: Time-based Expiry (Short Delay) ---
    logger.info("\n--- Experiment 2: Time-based Expiry (Short Delay) ---")
    loc_key_ex2 = "taviton_street" # Use a fresh pair from LOCATION_DATA
    details_ex2 = LOCATION_DATA[loc_key_ex2]
    original_encoding_ex2 = details_ex2["c3_encoding"]
    original_clienttime_ex2 = details_ex2["c3_clienttime"]

    # Initial use (should work)
    run_test_attempt(sdk, "Ex2.1: Initial Use (for delay test)", loc_key_ex2, original_encoding_ex2, original_clienttime_ex2, expected_to_succeed=True)
    
    delay_seconds = 60 * 2 # 2 minutes
    logger.info(f"Waiting for {delay_seconds // 60} minutes before retrying...")
    time.sleep(delay_seconds)
    
    # Retry after delay with original tokens
    # Expected to FAIL if tokens expire within this timeframe or are single use.
    run_test_attempt(sdk, f"Ex2.2: Reuse after {delay_seconds // 60} min delay (Original Tokens)", loc_key_ex2, original_encoding_ex2, original_clienttime_ex2, expected_to_succeed=False)

    # Retry after delay with original c3_encoding but FRESH c3_clienttime
    # This tests if c3_encoding is independently valid for longer if clienttime is fresh.
    # Highly likely to FAIL as they are probably a pair.
    fresh_clienttime = f"{time.time():.6f}"
    logger.info(f"Generated fresh c3_clienttime: {fresh_clienttime}")
    run_test_attempt(sdk, f"Ex2.3: Reuse after {delay_seconds // 60} min delay (Old Encoding, Fresh Time)", loc_key_ex2, original_encoding_ex2, fresh_clienttime, expected_to_succeed=False)


    # --- Experiment 3: "Tomorrow" Test (Simulated by using a known old pair) ---
    # This experiment is best run by actually scheduling the script.
    # For an immediate simulation, we assume the 'cromer_street' tokens are now "old"
    # if they were used in Experiment 1 and are single-use.
    logger.info("\n--- Experiment 3: 'Tomorrow' Test (Simulated) ---")
    logger.info("This simulates using a token pair that is assumed to be expired or already used.")
    # Re-using tokens from Ex1, which are likely invalid now if they were single-use.
    run_test_attempt(sdk, "Ex3.1: Simulated 'Tomorrow' (Reuse Old/Used Tokens)", loc_key_ex1, original_encoding_ex1, original_clienttime_ex1, expected_to_succeed=False)

    logger.info("\n" + "="*50 + "\nEXPIRY EXPERIMENTS CONCLUDED\n" + "="*50)


# --- Main execution example ---
if __name__ == "__main__":
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )
    
    sdk = TflCycleHireSDK()

    # --- Standard Run (Optional - can be commented out if only running experiments) ---
    logger.info("--- Standard SDK Run (using LOCATION_DATA values) ---")
    locations_to_try: list[LocationKey] = ["cromer_street", "taviton_street", "warren_street_station"]
    all_successful_standard_run = True
    for loc_key in locations_to_try:
        loc_details = LOCATION_DATA[loc_key]
        try:
            code = sdk.get_release_code(
                location_key=loc_key,
                c3_encoding=loc_details["c3_encoding"],
                c3_clienttime=loc_details["c3_clienttime"]
            )
            logger.info(f"Standard Run - SUCCESS! {loc_details['point_name']} Code: {code}")
        except TflCycleHireSDKError as e:
            logger.error(f"Standard Run - SDK Error for {loc_details['point_name']}: {e}")
            all_successful_standard_run = False
        except Exception as e:
            logger.error(f"Standard Run - Unexpected error for {loc_details['point_name']}: {e}", exc_info=True)
            all_successful_standard_run = False
        logger.info("-" * 30)
    if all_successful_standard_run:
        logger.info("Standard run completed successfully with example data.")
    else:
        logger.warning("Standard run encountered failures.")
    # --- End Standard Run ---

    # --- Perform Experiments ---
    perform_expiry_experiments(sdk)

    logger.info("\nTo perform a true 'Tomorrow' test:")
    logger.info("1. Ensure the 'Standard Run' section above works today.")
    logger.info("2. Comment out the call to 'perform_expiry_experiments(sdk)'.")
    logger.info("3. Schedule this script to run tomorrow without changing the LOCATION_DATA.")
    logger.info("4. Observe if the 'Standard Run' still succeeds (highly unlikely for all tokens).")

2025-05-29 02:27:28,763 - __main__ - INFO - --- Standard SDK Run (using LOCATION_DATA values) ---
2025-05-29 02:27:28,765 - __main__ - INFO - Requesting release code for: Cromer Street, Bloomsbury (cromer_street)
2025-05-29 02:27:30,332 - __main__ - INFO - Successfully extracted release code: 23313 for Cromer Street, Bloomsbury
2025-05-29 02:27:30,334 - __main__ - INFO - Standard Run - SUCCESS! Cromer Street, Bloomsbury Code: 23313
2025-05-29 02:27:30,335 - __main__ - INFO - ------------------------------
2025-05-29 02:27:30,335 - __main__ - INFO - Requesting release code for: Taviton Street, Bloomsbury (taviton_street)
2025-05-29 02:27:32,054 - __main__ - INFO - Successfully extracted release code: 21222 for Taviton Street, Bloomsbury
2025-05-29 02:27:32,055 - __main__ - INFO - Standard Run - SUCCESS! Taviton Street, Bloomsbury Code: 21222
2025-05-29 02:27:32,056 - __main__ - INFO - ------------------------------
2025-05-29 02:27:32,056 - __main__ - INFO - Requesting release code for: